# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('churn').getOrCreate()

In [2]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [3]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [15]:
data.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                 900|                 900|                900|
|   mean|         NULL|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|                NULL|                NULL|0.16666666666666666|
| stddev|         NULL|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.764835592035

In [5]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [8]:
from pyspark.ml.feature import VectorAssembler

In [20]:
assembler = VectorAssembler(inputCols=['Age',  'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

In [22]:
output = assembler.transform(data)

In [23]:
final_data = output.select('features', 'churn')

In [29]:
train_churn, test_churn = final_data.randomSplit([0.7,.3])

In [30]:
from pyspark.ml.classification import LogisticRegression

In [33]:
log_reg_churn = LogisticRegression(featuresCol='features',labelCol='churn')

In [34]:
final_fit = log_reg_churn.fit(train_churn)

In [38]:
training_sum = final_fit.summary
training_sum.predictions.describe().show()

+-------+------------------+-------------------+
|summary|             churn|         prediction|
+-------+------------------+-------------------+
|  count|               639|                639|
|   mean|0.1674491392801252|0.12519561815336464|
| stddev|0.3736688853634411| 0.3311998481529029|
|    min|               0.0|                0.0|
|    max|               1.0|                1.0|
+-------+------------------+-------------------+



In [39]:
pred_and_labels = final_fit.evaluate(test_churn)
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,1....|    0|[4.58541403416039...|[0.98990345402158...|       0.0|
|[25.0,9672.03,0.0...|    0|[5.06752531508434...|[0.99374142998485...|       0.0|
|[29.0,5900.78,1.0...|    0|[4.34408845360901...|[0.98718306877718...|       0.0|
|[29.0,9378.24,0.0...|    0|[5.11580892014072...|[0.99403467718419...|       0.0|
|[29.0,10203.18,1....|    0|[3.85318627074814...|[0.97922856323403...|       0.0|
|[30.0,6744.87,0.0...|    0|[3.86957995850244...|[0.97955940398219...|       0.0|
|[30.0,7960.64,1.0...|    1|[2.81178304855326...|[0.94330924652616...|       0.0|
|[31.0,5387.75,0.0...|    0|[3.13364695948184...|[0.95825950887397...|       0.0|
|[31.0,8688.21,0.0...|    0|[7.00198964788664...|[0.99909075802807...|       0.0|
|[31.0,8829.83,1

In [41]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [43]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='churn')

In [44]:
auc = churn_eval.evaluate(pred_and_labels.predictions)
auc

0.7723490505653937

0.7 Not bad, no so good👆

## Predict on new customers

In [46]:
final_lr_model = log_reg_churn.fit(final_data)

In [48]:
new_customers = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [49]:
new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [50]:
test_new_customers = assembler.transform(new_customers)

In [51]:
test_new_customers.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [52]:
final_results = final_lr_model.transform(test_new_customers)

In [54]:
final_results.select('Company', 'prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



1 gives prediction on churn, therefore recommendation that the company assigns a manager 👆